# Library


In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from datetime import datetime
from glob import glob
import pyexasol

In [2]:
# 소수점 옵션
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

# server

In [3]:
# # insider db
# wb = pyexasol.connect(dsn='172.29.135.35/F99FAB2444F86051A9A467F6313FAAB48AF7C4760663430958E3B89A9DC53361:8563', user='sys', password='exasol', compression=True, schema='VSYSD')
# exasol db
wb = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='vsyse')
wbt = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='wbt')

# fold path

In [4]:
drive = 'D'
ods_fold = f'{drive}:/data/big2/ODS'
raw1_fold = f'{drive}:/data/big2/BD1/raw'
df1_fold = f'{drive}:/data/big2/BD1/df'
an1_fold = f'{drive}:/data/big2/BD1/an'

# Load
- 등록정보&제원정보 -> result 4등급(연료 전체)
- 정기검사&정밀검사

## 등록정보(STD_CEG_CAR_MIG) 4등급만

In [5]:
# 8.6s
car = wb.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD FROM STD_CEG_CAR_MIG WHERE EXHST_GAS_GRD_CD = 'A0504' OR EXHST_GAS_GRD_CD = 'A05T4';")
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1898814 entries, 0 to 1898813
Data columns (total 10 columns):
 #   Column             Dtype 
---  ------             ----- 
 0   VIN                object
 1   BSPL_STDG_CD       int64 
 2   EXHST_GAS_GRD_CD   object
 3   EXHST_GAS_CERT_NO  object
 4   VHCL_ERSR_YN       object
 5   MANP_MNG_NO        object
 6   YRIDNW             int64 
 7   VHCTY_CD           object
 8   PURPS_CD2          object
 9   FRST_REG_YMD       int64 
dtypes: int64(3), object(7)
memory usage: 144.9+ MB


In [6]:
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소YN',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
}

In [7]:
carr = car.rename(columns=car_ch_col)

## 제원정보(STD_CEG_CAR_SRC_MIG)

In [8]:
# 3.8s
src = wb.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY FROM STD_CEG_CAR_SRC_MIG;")
src.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561153 entries, 0 to 561152
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   MANP_MNG_NO   561153 non-null  object
 1   FUEL_CD       518235 non-null  object
 2   VHCTY_TY_CD2  546198 non-null  object
 3   MNFCTR_NM     544020 non-null  object
 4   VHCNM         561051 non-null  object
 5   VHCL_FRM      531308 non-null  object
 6   EGIN_TY       508601 non-null  object
dtypes: object(7)
memory usage: 30.0+ MB


In [9]:
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
}

In [10]:
srcr = src.rename(columns=src_ch_col)

## 정기검사(STD_TB_JGT_HIS)

In [11]:
# 3m 34.9s
# jgt = wb.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_JT_YN1, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1 FROM STD_TB_JGT_HIS WHERE ROWNUM <= 10000;") # 테스트용
jgt = wb.export_to_pandas("SELECT VIN, FDRM_INSP_INSP_MTHD_CD, FDRM_INSP_KND_CD, FDRM_INSP_JGMT, FDRM_NLOD_SMO_MSTVL1, FDRM_NLOD_SMO_MSTVL2, FDRM_NLOD_SMO_MSTVL3, FDRM_NLOD_SMO_JT_YN1, FDRM_INSP_YMD, FDRM_DRVNG_DSTNC, FDRM_NLOD_SMO_PRMVL1 FROM STD_TB_JGT_HIS;")
jgt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64316863 entries, 0 to 64316862
Data columns (total 11 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   VIN                     object 
 1   FDRM_INSP_INSP_MTHD_CD  object 
 2   FDRM_INSP_KND_CD        object 
 3   FDRM_INSP_JGMT          object 
 4   FDRM_NLOD_SMO_MSTVL1    float64
 5   FDRM_NLOD_SMO_MSTVL2    float64
 6   FDRM_NLOD_SMO_MSTVL3    float64
 7   FDRM_NLOD_SMO_JT_YN1    object 
 8   FDRM_INSP_YMD           int64  
 9   FDRM_DRVNG_DSTNC        float64
 10  FDRM_NLOD_SMO_PRMVL1    float64
dtypes: float64(5), int64(1), object(5)
memory usage: 5.3+ GB


In [12]:
jgt_ch_col = {
    'VIN':'차대번호', 
    'FDRM_INSP_INSP_MTHD_CD':'검사방법', 
    'FDRM_INSP_KND_CD':'검사종류', 
    'FDRM_INSP_JGMT':'검사판정', 
    'FDRM_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'FDRM_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'FDRM_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'FDRM_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'FDRM_INSP_YMD':'검사일자',
    'FDRM_DRVNG_DSTNC':'주행거리',
    'FDRM_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}

In [13]:
jgtr = jgt.rename(columns=jgt_ch_col)
jgtr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

## 정밀검사(STD_TB_EET_HIS_ME)

In [14]:
# 6m 36.1s
# eet = wb.export_to_pandas("SELECT VIN, PRCINSP_MSRMT_MTHD_CD, PRCINSP_KND_CD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PREC_NLOD_SMO_JT_YN1, PRCINSP_YMD, PRCINSP_DRVNG_DSTNC FROM STD_TB_EET_HIS_ME WHERE ROWNUM <= 10000;") # 테스트용
eet = wb.export_to_pandas("SELECT VIN, PRCINSP_MSRMT_MTHD_CD, PRCINSP_KND_CD, PRCINSP_JGMT, PREC_NLOD_SMO_MSTVL1, PREC_NLOD_SMO_MSTVL2, PREC_NLOD_SMO_MSTVL3, PREC_NLOD_SMO_JT_YN1, PRCINSP_YMD, PRCINSP_DRVNG_DSTNC, PREC_NLOD_SMO_PRMVL1 FROM STD_TB_EET_HIS_ME;")
eet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99608917 entries, 0 to 99608916
Data columns (total 11 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   VIN                    object 
 1   PRCINSP_MSRMT_MTHD_CD  object 
 2   PRCINSP_KND_CD         object 
 3   PRCINSP_JGMT           object 
 4   PREC_NLOD_SMO_MSTVL1   float64
 5   PREC_NLOD_SMO_MSTVL2   float64
 6   PREC_NLOD_SMO_MSTVL3   float64
 7   PREC_NLOD_SMO_JT_YN1   object 
 8   PRCINSP_YMD            int64  
 9   PRCINSP_DRVNG_DSTNC    float64
 10  PREC_NLOD_SMO_PRMVL1   float64
dtypes: float64(5), int64(1), object(5)
memory usage: 8.2+ GB


In [15]:
eet.columns

Index(['VIN', 'PRCINSP_MSRMT_MTHD_CD', 'PRCINSP_KND_CD', 'PRCINSP_JGMT',
       'PREC_NLOD_SMO_MSTVL1', 'PREC_NLOD_SMO_MSTVL2', 'PREC_NLOD_SMO_MSTVL3',
       'PREC_NLOD_SMO_JT_YN1', 'PRCINSP_YMD', 'PRCINSP_DRVNG_DSTNC',
       'PREC_NLOD_SMO_PRMVL1'],
      dtype='object')

In [16]:
eet_ch_col = {
    'VIN':'차대번호', 
    'PRCINSP_MSRMT_MTHD_CD':'검사방법', 
    'PRCINSP_KND_CD':'검사종류', 
    'PRCINSP_JGMT':'검사판정', 
    'PREC_NLOD_SMO_MSTVL1':'무부하매연측정치1', 
    'PREC_NLOD_SMO_MSTVL2':'무부하매연측정치2', 
    'PREC_NLOD_SMO_MSTVL3':'무부하매연측정치3', 
    'PREC_NLOD_SMO_JT_YN1':'무부하매연판정1', 
    'PRCINSP_YMD':'검사일자',
    'PRCINSP_DRVNG_DSTNC':'주행거리', 
    'PREC_NLOD_SMO_PRMVL1':'무부하매연허용치1', 
}

In [17]:
eetr = eet.rename(columns=eet_ch_col)
eetr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

## 법정동코드(STD_BJCD_INFO)

In [18]:
# 1.3s
code = wb.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM FROM STD_BJCD_INFO;")
code.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46333 entries, 0 to 46332
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   STDG_CD       46333 non-null  int64 
 1   STDG_CTPV_NM  46333 non-null  object
 2   STDG_SGG_NM   46310 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [19]:
code.columns

Index(['STDG_CD', 'STDG_CTPV_NM', 'STDG_SGG_NM'], dtype='object')

In [20]:
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구',  
}

In [21]:
coder = code.rename(columns=code_ch_col)
coder.columns

Index(['법정동코드', '시도', '시군구'], dtype='object')

## RH에서 제공한 법정동코드

In [22]:
rh = pd.read_excel(os.path.join(raw1_fold, 'tb_mapdata_gu(2023.04.20).xlsx'))
rh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   dong_code    278 non-null    int64  
 1   ctprvn_code  278 non-null    int64  
 2   signgu_code  278 non-null    int64  
 3   sdstrd_code  278 non-null    int64  
 4   ctprvn_nm    278 non-null    object 
 5   signgu_nm    260 non-null    object 
 6   sdstrd_nm    0 non-null      float64
dtypes: float64(1), int64(4), object(2)
memory usage: 15.3+ KB


In [23]:
rh = rh.rename(columns={'dong_code':'법정동코드_rh', 'ctprvn_nm':'시도', 'signgu_nm':'시군구'})
rh.head()

,법정동코드_rh,ctprvn_code,signgu_code,sdstrd_code,시도,시군구,sdstrd_nm
0,0,0,0,0,전국,NaN,NaN
1,1100000000,11,0,0,서울특별시,NaN,NaN
2,1111000000,11,110,0,서울특별시,종로구,NaN
3,1114000000,11,140,0,서울특별시,중구,NaN
4,1117000000,11,170,0,서울특별시,용산구,NaN


## 4등급 result(for DPF유무)

In [24]:
rs = pd.read_csv(os.path.join(raw1_fold, '4등급_차량_리스트_양식_result_최종(전체)(2023.07.03).csv'), usecols=['차대번호', 'DPF유무_수정'])
rs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1502235 entries, 0 to 1502234
Data columns (total 2 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   차대번호      1502235 non-null  object
 1   DPF유무_수정  1153813 non-null  object
dtypes: object(2)
memory usage: 22.9+ MB


C:\Users\kbjung\AppData\Local\Temp\ipykernel_7812\865979260.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  rs = pd.read_csv(os.path.join(raw1_fold, '4등급_차량_리스트_양식_result_최종(전체)(2023.07.03).csv'), usecols=['차대번호', 'DPF유무_수정'])


# 전처리

## 중복 차대번호 제거

In [25]:
carr.shape, len(carr['차대번호'].unique())

((1898814, 10), 1898763)

In [26]:
carr = carr.sort_values('최초등록일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
carr.shape

(1898763, 10)

## 배출가스등급 코드 변환

In [27]:
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
carr['배출가스등급'].unique()

array(['4'], dtype=object)

## 차종 코드 변환

In [28]:
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
carr['차종'].unique()

array(['화물', '승합', '승용', '특수', nan], dtype=object)

## 용도 코드 변환

In [29]:
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
carr['용도'].unique()

array(['개인용', '영업용', '관용'], dtype=object)

## 등록정보 말소 제거

In [30]:
carm = carr[carr['차량말소YN'] == 'N'].reset_index(drop=True)
carm.shape

(1515737, 10)

## 연료 코드 변환

In [31]:
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
srcr['연료'].unique()

array(['경유', '휘발유', 'CNG(압축천연가스)', nan, 'LPG(액화석유가스)', '기타연료',
       '휘발유 하이브리드', '알코올', '전기', '태양열', 'CNG 하이브리드', '등유', '경유 하이브리드',
       'LPG 하이브리드', '수소', 'LNG(액화천연가스)'], dtype=object)

## 차종유형 코드 변환

In [32]:
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
srcr['차종유형'].unique()

array(['대형', '소형', '중형', '경형', nan], dtype=object)

## 등록&제원 병합

In [33]:
# 0.7s
cs = carm.merge(srcr, on='제원관리번호', how='left')
cs.shape

(1515737, 16)

In [34]:
cs.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,연료,차종유형,제작사명,차명,자동차형식,엔진형식
0,KMCLK18KP8C035195,4824034000,4,NaN,N,A0810005209353308,2008,화물,개인용,20220614,경유,대형,현대자동차,메가트럭군물자수송,HD050-8MT2-LGc-A,D6GA
1,KMFLA18KP8C031274,4482533000,4,NaN,N,B1R10005100003308,2008,화물,개인용,20220419,경유,대형,신정개발특장차,신정4㎥급진공노면총소차,RS07GMS2,D6GA
2,KMCGK17HP9C112030,4518010400,4,7MY-HD-24-28,N,A0810005008143209,2009,화물,개인용,20220407,경유,중형,현대자동차,마이티군물자수송트럭,HD25-9MT0-LHFbF,D4GA
3,KLACA26RD8B226753,2826011300,4,NaN,N,A0710001100361207,2008,승용,개인용,20210909,경유,중형,한국지엠,윈스톰2.0S디젤,CA26R,Z20S
4,KNCSE01429K388374,2771026522,4,7MY-KM-14-65,N,A0110003115353108,2009,화물,개인용,20210909,경유,소형,기아자동차,봉고Ⅲ 1톤,SET42K-A,J3


## 정기&정밀 병합

In [35]:
jgtr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

In [36]:
eetr.columns

Index(['차대번호', '검사방법', '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3',
       '무부하매연판정1', '검사일자', '주행거리', '무부하매연허용치1'],
      dtype='object')

In [37]:
jgtr['정기정밀'] = '정기'
eetr['정기정밀'] = '정밀'

In [38]:
# 3m 1.9s
ins = pd.concat([jgtr, eetr], ignore_index=True)
ins.shape

(163925780, 12)

### 최근 검사만 활용

In [39]:
# 4m 14.3s
insm = ins.sort_values('검사일자', ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
insm.shape

(34114272, 12)

## 검사종류 코드 변환

In [40]:
# 27.5s
knd_dict = {
    'A21P01':'정밀검사', 
    'A1904':'정기검사', 
    'A21T06':'종합검사(경과)', 
    'A21T05':'종합검사', 
    'A21S01':'특정검사', 
    'A1902':'재검사(2부재검사)',
    'A21R02':'2부재검사(종합)', 
    'A21P02':'정밀검사(경과)', 
    'A1903':'정기(경과)검사', 
    'A21S04':'특정재검사(배출)', 
    'A21E01':'배출재검사', 
    'A21R01':'1부재검사(종합)',
    'A21C02':'구조변경검사', 
    'A21002':'관능재검사(임시)', 
    'A21S03':'특정재검사(관능)', 
    'A21S02':'특정검사(경과)', 
    'A1901':'재검사(1부재검사)', 
    'A21T02':'종합(정밀)',
    'A21E02':'배출재검사(구변)', 
    'A21T01':'종합(경과:정밀)', 
    'A21T04':'종합(정밀) 2부재검사', 
    'A21001':'관능재검사', 
    'A21T03':'종합(정밀) 1부재검사', 
    'A21TMP':'임시검사', 
}
insm['검사종류'] = insm['검사종류'].replace(knd_dict)
insm['검사종류'].unique()

array(['종합검사', '종합검사(경과)', '1부재검사(종합)', '2부재검사(종합)', '정기검사', '재검사(1부재검사)',
       '정기(경과)검사', '재검사(2부재검사)', nan, '배출재검사(구변)', '구조변경검사',
       '종합(정밀) 1부재검사', '종합(정밀) 2부재검사', '종합(정밀)', '종합(경과:정밀)', '정밀검사',
       'A21C01', '특정재검사(배출)', '임시검사', '배출재검사', '관능재검사', '특정재검사(관능)',
       '특정검사', '특정검사(경과)', '정밀검사(경과)', 'A21R05', 'A21R06', '관능재검사(임시)',
       'A21E03'], dtype=object)

## 검사방법 코드 변환

In [41]:
# 14.9s
mth_dict = {
    'A18A':'무부하검사(TSI)', 
    'A18B':'무부하검사(급가속)', 
    'A18C':'무부하검사(정지가동)',
    'A2301':'무부하검사(급가속)', 
    'A2302':'무부하검사(정지가동)',
    'A2303':'무부하검사(TSI)', 
    'A2304':'부하검사(LUG DOWN)', 
    'A2305':'부하검사(ASM-Idling)', 
    'A2306':'부하검사(KD-147)', 
    'A2307':'부하검사(ASM2525)', 
}
insm['검사방법'] = insm['검사방법'].replace(mth_dict)
insm['검사방법'].unique()

array(['무부하검사(급가속)', '부하검사(KD-147)', '부하검사(ASM-Idling)', '부하검사(LUG DOWN)',
       '무부하검사(TSI)', '무부하검사(정지가동)', '부하검사(ASM2525)'], dtype=object)

## 등록&제원&정기&정밀 병합

In [42]:
# 2m 0.5s
csi = cs.merge(insm, on='차대번호', how='left')
csi.shape

(1515737, 27)

In [43]:
csi.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '검사방법',
       '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1',
       '검사일자', '주행거리', '무부하매연허용치1', '정기정밀'],
      dtype='object')

In [44]:
csi.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,...,검사종류,검사판정,무부하매연측정치1,무부하매연측정치2,무부하매연측정치3,무부하매연판정1,검사일자,주행거리,무부하매연허용치1,정기정밀
0,KMCLK18KP8C035195,4824034000,4,NaN,N,A0810005209353308,2008,화물,개인용,20220614,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KMFLA18KP8C031274,4482533000,4,NaN,N,B1R10005100003308,2008,화물,개인용,20220419,...,종합검사,Y,7.00,1.00,0.00,Y,20230330.00,4774.00,15.00,정밀
2,KMCGK17HP9C112030,4518010400,4,7MY-HD-24-28,N,A0810005008143209,2009,화물,개인용,20220407,...,정기(경과)검사,Y,5.00,NaN,NaN,Y,20230523.00,79425.00,20.00,정기
3,KLACA26RD8B226753,2826011300,4,NaN,N,A0710001100361207,2008,승용,개인용,20210909,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,KNCSE01429K388374,2771026522,4,7MY-KM-14-65,N,A0110003115353108,2009,화물,개인용,20210909,...,2부재검사(종합),Y,14.00,NaN,NaN,Y,20220810.00,50589.00,15.00,정밀


## 등록&제원&정기&정밀&법정동코드 병합

In [45]:
csi.shape

(1515737, 27)

In [46]:
df = csi.merge(coder, on='법정동코드', how='left')

In [47]:
df.shape

(1515737, 29)

In [48]:
df['시도'].isnull().sum()

1

In [49]:
df.loc[df['시도'].isnull(), '법정동코드']

1403555    4163055000
Name: 법정동코드, dtype: int64

In [50]:
# 주소 수정
df.loc[df['법정동코드'] == 4163055000, ['시도', '시군구']] = ['경기도', '양주시'] # 경기도 양주시 회천3동

## rh 법정동코드 참고하여 법정동코드 수정

In [51]:
df.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '검사방법',
       '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1',
       '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구'],
      dtype='object')

In [52]:
rdf = df.copy()

In [53]:
rdf['법정동코드'] = rdf['법정동코드'].astype('str')
rdf['법정동코드_mod'] = rdf['법정동코드'].str[:5] + '00000'
rdf['법정동코드_mod'] = pd.to_numeric(rdf['법정동코드_mod'])
rdf['법정동코드_mod'].head()

0    4824000000
1    4482500000
2    4518000000
3    2826000000
4    2771000000
Name: 법정동코드_mod, dtype: int64

In [54]:
set(rdf['법정동코드_mod'].unique()) - set(rh['법정동코드_rh'].unique())

{2817000000, 4119500000, 4119700000, 4119900000, 4173000000, 4371000000}

In [55]:
# 2817000000 인천광역시 남구 -> 인천광역시 미추홀구 2817700000
# 4119500000 경기도 부천시 원미구 -> 경기도 부천시 4119000000
# 4119700000 경기도 부천시 소사구 -> 경기도 부천시 4119000000
# 4119900000 경기도 부천시 오정구 -> 경기도 부천시 4119000000
# 4173000000 경기도 여주군 -> 경기도 여주시 4167000000
# 4371000000 충청북도 청원군 -> 충청북도 충주시 4311000000

In [56]:
rdf.loc[rdf['법정동코드_mod'] == 2817000000, '법정동코드_mod'] = 2817700000
rdf.loc[rdf['법정동코드_mod'] == 4119500000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4119700000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4119900000, '법정동코드_mod'] = 4119000000
rdf.loc[rdf['법정동코드_mod'] == 4173000000, '법정동코드_mod'] = 4167000000
rdf.loc[rdf['법정동코드_mod'] == 4371000000, '법정동코드_mod'] = 4311000000

In [57]:
set(rdf['법정동코드_mod'].unique()) - set(rh['법정동코드_rh'].unique())

set()

In [58]:
rdf.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '검사방법',
       '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1',
       '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구', '법정동코드_mod'],
      dtype='object')

In [59]:
rdf['법정동코드_mod'].isnull().sum()

0

In [60]:
df = rdf.copy()
df.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '검사방법',
       '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1',
       '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구', '법정동코드_mod'],
      dtype='object')

## 4등급 result 파일 참고하여 DPF유무 수정

In [61]:
rdf = df.copy()

In [62]:
len(set(rdf['차대번호'].unique()) - set(rs['차대번호'].unique()))

53185

In [64]:
rs['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
무       879481
NaN     348422
유       261897
확인불가     12435
Name: count, dtype: int64

In [65]:
rs = rs.drop_duplicates('차대번호').reset_index(drop=True)
rs.shape

(1502213, 2)

In [66]:
rdf1 = rdf.merge(rs, on='차대번호', how='left')

In [67]:
rdf1['DPF유무_수정'].value_counts(dropna=False)

DPF유무_수정
무       839836
NaN     401579
유       261889
확인불가     12433
Name: count, dtype: int64

In [68]:
rdf1.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '검사방법',
       '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1',
       '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구', '법정동코드_mod',
       'DPF유무_수정'],
      dtype='object')

In [69]:
df = rdf1.copy()

In [70]:
df.shape

(1515737, 31)

# STD_BD_GRD4_CAR_CURSTT

## 4등급차만 추출

In [71]:
# 2.8s
df1 = df[df['배출가스등급'] == '4'].reset_index(drop=True)
df1.shape

(1515737, 31)

### 기준연월 정보 추가

In [73]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'202307'

In [74]:
df1['테이블생성일자'] = today_date

In [75]:
df1.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '검사방법',
       '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1',
       '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구', '법정동코드_mod',
       'DPF유무_수정', '기준연월'],
      dtype='object')

In [76]:
STD_BD_GRD4_CAR_CURSTT = df1[[
    '테이블생성일자', 
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '용도',
    '차량연식', 
    '차종유형', 
    '연료', 
    '법정동코드', 
    '시도',
    '시군구',
    '차명', 
    '제작사명', 
    '배출가스등급', 
    'DPF유무_수정',
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '무부하매연측정치1', 
    '무부하매연판정1', 
    '무부하매연측정치2', 
    '무부하매연측정치3', 
    '법정동코드_mod', 
    ]]
STD_BD_GRD4_CAR_CURSTT.shape

(1515737, 23)

In [77]:
STD_BD_GRD4_CAR_CURSTT.head()

,기준연월,차대번호,제원관리번호,차종,용도,차량연식,차종유형,연료,법정동코드,시도,...,배출가스등급,DPF유무_수정,검사방법,검사종류,검사판정,무부하매연측정치1,무부하매연판정1,무부하매연측정치2,무부하매연측정치3,법정동코드_mod
0,202307,KMCLK18KP8C035195,A0810005209353308,화물,개인용,2008,대형,경유,4824034000,경상남도,...,4,유,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4824000000
1,202307,KMFLA18KP8C031274,B1R10005100003308,화물,개인용,2008,대형,경유,4482533000,충청남도,...,4,확인불가,부하검사(LUG DOWN),종합검사,Y,7.00,Y,1.00,0.00,4482500000
2,202307,KMCGK17HP9C112030,A0810005008143209,화물,개인용,2009,중형,경유,4518010400,전라북도,...,4,유,무부하검사(급가속),정기(경과)검사,Y,5.00,Y,NaN,NaN,4518000000
3,202307,KLACA26RD8B226753,A0710001100361207,승용,개인용,2008,중형,경유,2826011300,인천광역시,...,4,유,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2826000000
4,202307,KNCSE01429K388374,A0110003115353108,화물,개인용,2009,소형,경유,2771026522,대구광역시,...,4,무,부하검사(KD-147),2부재검사(종합),Y,14.00,Y,NaN,NaN,2771000000


In [78]:
ch_col_dict = {
                '테이블생성일자':'LOAD_DT',
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO', 
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2', 
                '차량연식':'YRIDNW', 
                '차종유형':'VHCTY_TY', 
                '연료':'FUEL_CD', 
                '법정동코드':'STDG_CD', 
                '시도':'CTPV_NM', 
                '시군구':'SGG_NM', 
                '차명':'VHCNM', 
                '제작사명':'MNFCTR_NM', 
                '배출가스등급':'EXHST_GAS_GRD_CD_MOD',
                'DPF유무_수정':'DPF_MNTNG_YN', 
                '검사방법':'INSP_MTHD', 
                '검사종류':'INSP_KND', 
                '검사판정':'INSP_JGMT', 
                '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
                '무부하매연판정1':'NOLOD_SMO_JGMT_YN1', 
                '무부하매연측정치2':'NOLOD_SMO_MEVLU2', 
                '무부하매연측정치3':'NOLOD_SMO_MEVLU3', 
                '법정동코드_mod':'STDG_CD_MOD',

                '제작일자':'FBCTN_YMD', 
                '차종분류':'VHCTY_CL_CD',
                }

In [79]:
STD_BD_GRD4_CAR_CURSTT = STD_BD_GRD4_CAR_CURSTT.rename(columns=ch_col_dict)
STD_BD_GRD4_CAR_CURSTT.columns

Index(['CRTR_YM', 'VIN', 'MANG_MNG_NO', 'VHCTY_CD', 'PURPS_CD2', 'YRIDNW',
       'VHCTY_TY', 'FUEL_CD', 'STDG_CD', 'CTPV_NM', 'SGG_NM', 'VHCNM',
       'MNFCTR_NM', 'EXHST_GAS_GRD_CD_MOD', 'DPF_MNTNG_YN', 'INSP_MTHD',
       'INSP_KND', 'INSP_JGMT', 'NOLOD_SMO_MEVLU1', 'NOLOD_SMO_JGMT_YN1',
       'NOLOD_SMO_MEVLU2', 'NOLOD_SMO_MEVLU3', 'STDG_CD_MOD'],
      dtype='object')

In [80]:
STD_BD_GRD4_CAR_CURSTT.shape

(1515737, 23)

## [출력] STD_BD_GRD4_CAR_CURSTT

In [81]:
expdf = STD_BD_GRD4_CAR_CURSTT
table_nm = 'STD_BD_GRD4_CAR_CURSTT'.upper()

# 테이블 생성
sql = 'create table ' + table_nm + '( \n'

for idx,column in enumerate(expdf.columns):
    if 'float' in expdf[column].dtype.name:
        sql += column + ' float'
    elif 'int' in expdf[column].dtype.name:
        sql += column + ' number'
    else:
        sql += column + ' varchar(255)'

    if len(expdf.columns) - 1 != idx:
        sql += ','
    sql += '\n'
sql += ')'    
we.execute(sql)

# 데이터 추가
# 9s
we.import_from_pandas(expdf, table_nm)

In [109]:
# about 8.2s
# 4등급 차량 현황
STD_BD_GRD4_CAR_CURSTT.to_csv(os.path.join(df1_fold,  f'STD_BD_GRD4_CAR_CURSTT.csv'), index=False)

# STD_BD_GRD4_CAR_CURSTT_TOT

In [70]:
df2 = df1.copy()

## 경유차만 추출

In [71]:
df2 = df2[df2['연료'] == '경유'].reset_index(drop=True)

## 차대번호 10자리 연식

In [72]:
df2['vin10'] = df2['차대번호'].str[9]

In [73]:
vin10_dict = {'J':1988, 'K':1989, 'L':1990, 'M':1991, 'N':1992, 'P':1993, 'R':1994, 'S':1995, 'T':1996, 'V':1997, 'W':1998, 'X':1999, 'Y':2000, '1':2001, '2':2002, '3':2003, '4':2004, '5':2005, '6':2006, '7':2007, '8':2008, '9':2009, 'A':2010, 'B':2011, 'C':2012, 'D':2013, 'E':2014, 'F':2015, 'G':2016, 'H':2017}

In [74]:
df2['vin10_year'] = df2['vin10'].map(vin10_dict, na_action='ignore')

## 배인번호_수정 문자 타입으로 변경

In [75]:
df2['배출가스인증번호'] = df2['배출가스인증번호'].astype('str')

## 차대번호 17자리 샘플

In [76]:
df2y = df2.loc[df2['차대번호'].str.len() == 17].reset_index(drop=True)
df2n = df2.loc[df2['차대번호'].str.len() != 17].reset_index(drop=True)
df2y.shape, df2n.shape

((1156229, 33), (10, 33))

## 차대번호 연식과 연식 동일한 샘플

In [77]:
df3y = df2y.loc[df2y['vin10_year'] == df2y['차량연식']].reset_index(drop=True)
df3n = df2y.loc[df2y['vin10_year'] != df2y['차량연식']].reset_index(drop=True)
df3y.shape, df3n.shape

((1155670, 33), (559, 33))

# 배인번호별 분석

In [78]:
def flat_cols(df):
    df.columns = ['/'.join(x) for x in df.columns.to_flat_index()]
    return df

In [79]:
# about 31.3s
# 최적화 24m 51s -> 30.0s
total_g_df = pd.DataFrame()
groupby_col1 = ['제작사명', '배출가스인증번호', '제원관리번호', '자동차형식', '엔진형식', '검사종류', '검사방법', '검사판정']
groupby_col2 = ['제작사명', '배출가스인증번호', '제원관리번호', '자동차형식', '엔진형식', '검사종류', '검사방법']
for one in tqdm(df3y['배출가스인증번호'].unique()):
    # 배인번호별 df
    gas_df = df3y.loc[df3y['배출가스인증번호'] == str(one)].reset_index(drop=True)

    if gas_df.shape[0] != 0:
        # 제번별, 차형식별, 엔진형식별, 검사판정별 무부하매연측정치1 통계
        g = gas_df.groupby(groupby_col1).agg({'차대번호':'count', '무부하매연측정치1':['mean', 'min', 'max']}).pipe(flat_cols).round(2).reset_index()
        g = g.rename(columns={'배출가스인증번호':'배출가스인증번호', '차대번호/count':'대수', '무부하매연측정치1/mean':'mean', '무부하매연측정치1/min':'min', '무부하매연측정치1/max':'max'})
        # 하나의 배인번호에서 제번별 엔진형식별 비율 계산
        g['합격률(%)'] = round(g['대수'] / g.groupby(groupby_col2)['대수'].transform('sum') * 100, 2)
        # 종합 - 통계
        total_g_df = pd.concat([total_g_df, g], ignore_index=True)
    else:
        print(f'오류 배인번호 : {one}')
        pass

  0%|          | 0/207 [00:00<?, ?it/s]

100%|██████████| 207/207 [00:16<00:00, 12.75it/s]


In [80]:
total_g_df.head()

,제작사명,배출가스인증번호,제원관리번호,자동차형식,엔진형식,검사종류,검사방법,검사판정,대수,mean,min,max,합격률(%)
0,SK 에너지울산COMPLEX,nan,2RH20000100003307,X036-H208,06R,종합검사,무부하검사(급가속),Y,1,18.00,18.00,18.00,100.00
1,SOFA차량,nan,99931107143071212,CA26R,Z20S,종합검사,부하검사(KD-147),Y,1,0.00,0.00,0.00,100.00
2,SOFA차량,nan,99931107151521312,R7DPD2-016,665925,종합검사,부하검사(KD-147),Y,1,15.00,15.00,15.00,100.00
3,SOFA차량,nan,99931107170911214,JE513D,D4EA,종합검사,부하검사(KD-147),Y,1,0.00,0.00,0.00,100.00
4,SOFA차량,nan,99931107186881215,JE513D,D4EA,종합검사,부하검사(KD-147),Y,1,6.00,6.00,6.00,100.00


# 분석

## 열화계수(SI) 지수 계산
- SI = 측정치 / 허용치

In [81]:
df3y.shape

(1155670, 33)

In [82]:
df3y.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '검사방법',
       '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1',
       '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구', '법정동코드_mod', '기준연월',
       'vin10', 'vin10_year'],
      dtype='object')

In [83]:
sidf = df3y[[
    '차대번호', 
    '제원관리번호', 
    '차종', 
    '차량연식', 
    '차명', 
    '차종유형', 
    '제작사명', 
    '연료', 
    '법정동코드', 
    '배출가스인증번호', 
    '검사방법', 
    '검사종류', 
    '검사판정', 
    '주행거리', 
    '무부하매연판정1', 
    '무부하매연허용치1', 
    '무부하매연측정치1'
    ]]
sidf.shape

(1155670, 17)

In [84]:
current_yr = int(datetime.today().strftime("%Y"))
current_yr

2023

In [85]:
sidf['차령'] = 2023 - sidf['차량연식']
sidf['SI'] = sidf['무부하매연측정치1'] / sidf['무부하매연허용치1']

C:\Users\kbjung\AppData\Local\Temp\ipykernel_19148\2774727232.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['차령'] = 2023 - sidf['차량연식']
C:\Users\kbjung\AppData\Local\Temp\ipykernel_19148\2774727232.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sidf['SI'] = sidf['무부하매연측정치1'] / sidf['무부하매연허용치1']


In [86]:
sidf[['차령', 'SI']].isnull().sum()

차령        0
SI    51367
dtype: int64

In [87]:
sidf[['무부하매연측정치1', '무부하매연허용치1']].isnull().sum()

무부하매연측정치1    14709
무부하매연허용치1    14712
dtype: int64

In [88]:
sidf.head()

,차대번호,제원관리번호,차종,차량연식,차명,차종유형,제작사명,연료,법정동코드,배출가스인증번호,검사방법,검사종류,검사판정,주행거리,무부하매연판정1,무부하매연허용치1,무부하매연측정치1,차령,SI
0,KMCLK18KP8C035195,A0810005209353308,화물,2008,메가트럭군물자수송,대형,현대자동차,경유,4824034000,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN
1,KMFLA18KP8C031274,B1R10005100003308,화물,2008,신정4㎥급진공노면총소차,대형,신정개발특장차,경유,4482533000,nan,부하검사(LUG DOWN),종합검사,Y,4774.00,Y,15.00,7.00,15,0.47
2,KMCGK17HP9C112030,A0810005008143209,화물,2009,마이티군물자수송트럭,중형,현대자동차,경유,4518010400,7MY-HD-24-28,무부하검사(급가속),정기(경과)검사,Y,79425.00,Y,20.00,5.00,14,0.25
3,KNCSE01429K388374,A0110003115353108,화물,2009,봉고Ⅲ 1톤,소형,기아자동차,경유,2771026522,7MY-KM-14-65,부하검사(KD-147),2부재검사(종합),Y,50589.00,Y,15.00,14.00,14,0.93
4,KLACA26RD8B226753,A0710001100361207,승용,2008,윈스톰2.0S디젤,중형,한국지엠,경유,2826011300,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,NaN


### 경유만 추출

In [89]:
sidf1 = sidf[sidf['연료'] == '경유'].reset_index(drop=True)

In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'202307'

In [ ]:
sidf1['테이블생성일자'] = today_date

In [90]:
sidf1 = sidf1[[
    '테이블생성일자'
    '차대번호', 
    '제원관리번호', 
    '차명', 
    '제작사명', 
    '배출가스인증번호', 
    '검사방법',
    '주행거리', 
    '차령',
    'SI', 
    ]]

In [91]:
chc_dict = {
    '테이블생성일자':'LOAD_DT'
    '차대번호':'VIN', 
    '제원관리번호':'MANG_MNG_NO', 
    '차종':'VHCTY_CD', 
    '연식':'YRIDNW', 
    '차명':'VHCNM',
    '제작사명':'MNFCTR_NM', 
    '차종유형':'VHCTY_TY', 
    '연료':'FUEL_CD',
    '법정동코드':'STDG_CD', 
    '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
    '검사방법':'INSP_MTHD', 
    '검사종류':'INSP_KND', 
    '검사판정':'INSP_JGMT', 
    '주행거리':'DRVNG_DSTNC',
    '무부하매연판정1':'NOLOD_SMO_JGMT_YN1',
    '무부하매연허용치1':'NOLOD_SMO_PRMT_VAL1',
    '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
    '차령':'VHCAG',
    }

In [92]:
STD_BD_GRD4_SI = sidf1.rename(columns=chc_dict)
STD_BD_GRD4_SI.columns

Index(['VIN', 'MANG_MNG_NO', 'VHCNM', 'MNFCTR_NM', 'EXHST_GAS_CERT_NO_MOD',
       'INSP_MTHD', 'DRVNG_DSTNC', 'VHCAG', 'SI'],
      dtype='object')

In [93]:
STD_BD_GRD4_SI.head()

,VIN,MANG_MNG_NO,VHCNM,MNFCTR_NM,EXHST_GAS_CERT_NO_MOD,INSP_MTHD,DRVNG_DSTNC,VHCAG,SI
0,KMCLK18KP8C035195,A0810005209353308,메가트럭군물자수송,현대자동차,nan,NaN,NaN,15,NaN
1,KMFLA18KP8C031274,B1R10005100003308,신정4㎥급진공노면총소차,신정개발특장차,nan,부하검사(LUG DOWN),4774.00,15,0.47
2,KMCGK17HP9C112030,A0810005008143209,마이티군물자수송트럭,현대자동차,7MY-HD-24-28,무부하검사(급가속),79425.00,14,0.25
3,KNCSE01429K388374,A0110003115353108,봉고Ⅲ 1톤,기아자동차,7MY-KM-14-65,부하검사(KD-147),50589.00,14,0.93
4,KLACA26RD8B226753,A0710001100361207,윈스톰2.0S디젤,한국지엠,nan,NaN,NaN,15,NaN


In [94]:
STD_BD_GRD4_SI.shape

(1155670, 9)

### [출력] SI 지수 정보(STD_BD_GRD4_SI)

In [ ]:
# expdf = STD_BD_GRD4_SI
# table_nm = 'STD_BD_GRD4_SI'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 6s
# we.import_from_pandas(expdf, table_nm)

In [86]:
# # about 9.0s
# # STD_BD_GRD4_SI
# # 4등급_경유차_SI지수
# STD_BD_GRD4_SI.to_csv(os.path.join(df1_fold, 'STD_BD_GRD4_SI.csv'), index=False)

## 동일 배번에서 제번별 매연 boxplot

In [95]:
total_g_df1 = total_g_df.loc[(total_g_df['배출가스인증번호'] != '확인불가') | (total_g_df['배출가스인증번호'] != 'nan')]
total_g_df1.shape

(30037, 13)

In [96]:
sample01 = total_g_df1.loc[total_g_df1['대수'] > 100].reset_index(drop=True)
sample01.shape

(1668, 13)

### quantile

In [97]:
# 4m 11.2s
boxplot_df = pd.DataFrame()
quantile_df = pd.DataFrame()
for one in sample01['배출가스인증번호'].unique(): # 통계에서 100대 초과인 배인번호만 추출
    temp_one = df3y.loc[(df3y['배출가스인증번호'] == one) & (df3y['검사판정'] == 'Y')].reset_index(drop=True) # 해당 배인번호 중 검사판정이 'Y'인 샘플만 추출
    if temp_one.shape[0] > 100: # 100대 초과면 진행
        for two in temp_one['검사종류'].unique(): # 검사종류별 샘플 추출
            temp_two = temp_one.loc[temp_one['검사종류'] == two].reset_index(drop=True)
            if temp_two.shape[0] > 100:
                for three in temp_two['검사방법'].unique(): # 검사방법별 샘플 추출
                    temp_three = temp_two.loc[temp_two['검사방법'] == three].reset_index(drop=True)
                    boxplot_df = pd.concat([boxplot_df, temp_three], ignore_index=True) # 해당 샘플만 추출하여 쌓기
                    if temp_three.shape[0] > 100:
                        xticks_list= []
                        data_list = []
                        for four in temp_three['제원관리번호'].unique():
                            temp_four = temp_three.loc[temp_three['제원관리번호'] == four].reset_index(drop=True).dropna(subset=['무부하매연측정치1'])
                            if temp_four.shape[0] > 100:
                                xticks_list.append(four)
                                data_list.append(temp_four['무부하매연측정치1'])
                                temp_four['q1'] = temp_four['무부하매연측정치1'].describe()['25%']
                                temp_four['q2'] = temp_four['무부하매연측정치1'].describe()['50%']
                                temp_four['q3'] = temp_four['무부하매연측정치1'].describe()['75%']
                                temp_four['차량대수'] = temp_four.shape[0]
                                quantile_df = pd.concat([quantile_df, temp_four], ignore_index=True) # 제번별 4분위 값 df형태로 저장

In [98]:
quantile_df.shape

(878322, 37)

In [99]:
quantile_df.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소YN', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '연료', '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '검사방법',
       '검사종류', '검사판정', '무부하매연측정치1', '무부하매연측정치2', '무부하매연측정치3', '무부하매연판정1',
       '검사일자', '주행거리', '무부하매연허용치1', '정기정밀', '시도', '시군구', '법정동코드_mod', '기준연월',
       'vin10', 'vin10_year', 'q1', 'q2', 'q3', '차량대수'],
      dtype='object')

In [100]:
quantile_df1 = quantile_df.drop_duplicates(['배출가스인증번호', '제원관리번호', '검사방법', '검사종류'])
quantile_df1.shape

(1531, 37)

In [101]:
quantile_df1.head()

,차대번호,법정동코드,배출가스등급,배출가스인증번호,차량말소YN,제원관리번호,차량연식,차종,용도,최초등록일자,...,시도,시군구,법정동코드_mod,기준연월,vin10,vin10_year,q1,q2,q3,차량대수
0,VF30U9HZHBS006764,4136011100,4,nan,N,00620000900021210,2011,승용,개인용,20110329,...,경기도,남양주시,4136000000,202307,B,2011.00,1.00,5.00,10.00,225
225,VF34C9HZHAS010877,4161025000,4,nan,N,00620000800081209,2010,승용,개인용,20100818,...,경기도,광주시,4161000000,202307,A,2010.00,1.00,5.00,10.00,211
436,VF34HRHRJ9S003363,4161011100,4,nan,N,00620000800021208,2009,승용,개인용,20100623,...,경기도,광주시,4161000000,202307,9,2009.00,3.00,6.00,10.00,127
563,VF33HRHRJ8S007818,2617010300,4,nan,N,00620000400221207,2008,승용,개인용,20100405,...,부산광역시,동구,2617000000,202307,8,2008.00,2.00,5.50,9.00,164
727,VF36DRHRJ8L004972,4157036023,4,nan,N,00620000500251207,2008,승용,개인용,20091102,...,경기도,김포시,4157000000,202307,8,2008.00,1.00,4.00,9.00,218


In [ ]:
today_date = datetime.today().strftime("%Y%m%d")
today_date

'202307'

In [ ]:
quantile_df1['테이블생성일자'] = today_date

In [102]:
quantile_df2 = quantile_df1[[
    '테이블생성일자',
    '차명',
    '제작사명', 
    '제원관리번호', 
    '배출가스인증번호', 
    '검사방법', 
    'q1', 
    'q2', 
    'q3',
    '차량대수',
    ]]
quantile_df2.shape

(1531, 9)

In [103]:
chc_dict = {
                '테이블생성일자':'LOAD_DT', 
                '차대번호':'VIN', 
                '제원관리번호':'MANG_MNG_NO', 
                '차명':'VHCNM',
                '제작사명':'MNFCTR_NM', 
                '차종':'VHCTY_CD', 
                '용도':'PURPS_CD2', 
                '차종유형':'CHCTY_TY', 
                '법정동코드':'STDG_CD', 
                '배출가스인증번호':'EXHST_GAS_CERT_NO_MOD', 
                '검사방법':'INSP_MTHD', 
                '검사종류':'INSP_KND', 
                '검사판정':'INSP_JGMT', 
                '무부하매연측정치1':'NOLOD_SMO_MEVLU1', 
                '무부하매연판정1':'NOLOD_SMO_JGMT_YN1',
                'q1':'LOWR_QRT',
                'q2':'MID_QRT',
                'q3':'UP_QRT',
                '차량대수':'VHCL_MKCNT',
                }
# '등급_수정':'EXHST_GAS_GRD_CD_MOD', 
# 'DPF유무_수정':'DPF_MNTNG_YN', 
# '시도명':'CTPV_NM', 
# '시군구명':'SGG_NM', 
# '차종분류':'VHCTY_CL_CD', 

In [104]:
STD_BD_GRD4_CAR_CURSTT_TOT = quantile_df2.rename(columns=chc_dict)
STD_BD_GRD4_CAR_CURSTT_TOT.columns

Index(['VHCNM', 'MNFCTR_NM', 'MANG_MNG_NO', 'EXHST_GAS_CERT_NO_MOD',
       'INSP_MTHD', 'LOWR_QRT', 'MID_QRT', 'UP_QRT', 'VHCL_MKCNT'],
      dtype='object')

In [105]:
STD_BD_GRD4_CAR_CURSTT_TOT.shape

(1531, 9)

In [106]:
STD_BD_GRD4_CAR_CURSTT_TOT.head()

,VHCNM,MNFCTR_NM,MANG_MNG_NO,EXHST_GAS_CERT_NO_MOD,INSP_MTHD,LOWR_QRT,MID_QRT,UP_QRT,VHCL_MKCNT
0,3008 1.6 HDi MCP,한불모터스,00620000900021210,nan,부하검사(KD-147),1.00,5.00,10.00,225
225,308 1.6 HDi MCP,한불모터스,00620000800081209,nan,부하검사(KD-147),1.00,5.00,10.00,211
436,Nouvelle Peugeot 308SW HDi,한불모터스,00620000800021208,nan,부하검사(KD-147),3.00,6.00,10.00,127
563,푸조 NEW 307SW HDi,한불모터스,00620000400221207,nan,부하검사(KD-147),2.00,5.50,9.00,164
727,푸조 407 HDi FAP A6,한불모터스,00620000500251207,nan,부하검사(KD-147),1.00,4.00,9.00,218


### [출력] 제번별 4분위 값 df(STD_BD_GRD4_CAR_CURSTT_TOT)

In [ ]:
# expdf = STD_BD_GRD4_CAR_CURSTT_TOT
# table_nm = 'STD_BD_GRD4_CAR_CURSTT_TOT'.upper()

# # 테이블 생성
# sql = 'create table ' + table_nm + '( \n'

# for idx,column in enumerate(expdf.columns):
#     if 'float' in expdf[column].dtype.name:
#         sql += column + ' float'
#     elif 'int' in expdf[column].dtype.name:
#         sql += column + ' number'
#     else:
#         sql += column + ' varchar(255)'

#     if len(expdf.columns) - 1 != idx:
#         sql += ','
#     sql += '\n'
# sql += ')'    
# wbt.execute(sql)

# # 데이터 추가
# # 1s
# we.import_from_pandas(expdf, table_nm)

In [131]:
# # about 5.9s
# # STD_BD_GRD4_CAR_CURSTT_TOT
# # 4등급_경유_제번별_사분위수
# quantile_df2.to_csv(os.path.join(df1_fold, f'STD_BD_GRD4_CAR_CURSTT_TOT.csv'), index=False)

# code end